Code 

3D wheel

In [3]:
# Parameters
vf =  0.3
γ_evo = 0.1 
max_steps = 10
α_coeff = γ_evo*max_steps
iter_mod = 50
D = 3
mesh_name = "Wheel_3d.msh"
mesh_file = (@__DIR__)*"/Models/$mesh_name"

"c:\\Users\\IIT BBSR\\Desktop\\Amiya\\BTP\\Julia\\Level_Set_Method\\Cut_FEM\\3D_Wheel/Models/Wheel_3d.msh"

In [4]:
# Output Path
path = "./Result/3D_Wheel"

#  Directory to store visualization file
files_path = path*"data/"
isdir(files_path) || mkpath(files_path)

#  Directory to store optimized model
model_path = path*"model/"
isdir(model_path) || mkpath(model_path)

"./Result/3D_Wheelmodel"

In [5]:
using GridapGmsh.GridapGmsh: GmshDiscreteModel
using Gridap.Geometry: UnstructuredDiscreteModel
using GridapTopOpt: update_labels!
using Gridap.Visualization: writevtk

In [6]:
# Load mesh
model = GmshDiscreteModel(mesh_file)
model = UnstructuredDiscreteModel(model)
f_diri(x) =
((cos(30π/180)<=x[1]<=cos(15π/180)) && abs(x[2] - sqrt(1-x[1]^2))<1e-4) ||
((cos(97.5π/180)<=x[1]<=cos(82.5π/180)) && abs(x[2] - sqrt(1-x[1]^2))<1e-4) ||
((cos(165π/180)<=x[1]<=cos(150π/180)) && abs(x[2] - sqrt(1-x[1]^2))<1e-4) ||
((cos(142.5π/180)<=x[1]<=cos(127.5π/180)) && abs(x[2] - -sqrt(1-x[1]^2))<1e-4) ||
((cos(52.5π/180)<=x[1]<=cos(37.5π/180)) && abs(x[2] - -sqrt(1-x[1]^2))<1e-4)
update_labels!(1,model,f_diri,"Gamma_D_new")
writevtk(model,model_path*"model")

Info    : Reading 'c:\Users\IIT BBSR\Desktop\Amiya\BTP\Julia\Level_Set_Method\Cut_FEM\3D_Wheel/Models/Wheel_3d.msh'...
Info    : 15 entities
Info    : 219664 nodes
Info    : 1328560 elements                                                                                    
Info    : Done reading 'c:\Users\IIT BBSR\Desktop\Amiya\BTP\Julia\Level_Set_Method\Cut_FEM\3D_Wheel/Models/Wheel_3d.msh'


4-element Vector{Vector{String}}:
 ["./Result/3D_Wheelmodel/model_0.vtu"]
 ["./Result/3D_Wheelmodel/model_1.vtu"]
 ["./Result/3D_Wheelmodel/model_2.vtu"]
 ["./Result/3D_Wheelmodel/model_3.vtu"]

In [7]:
using Gridap.Geometry: Triangulation
using GridapTopOpt: get_element_diameters

In [8]:
# Get triangulation and element size
Ω_bg = Triangulation(model)
hₕ = get_element_diameter_field(model)
hmin = minimum(get_element_diameters(model))

UndefVarError: UndefVarError: `get_element_diameter_field` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing GridapTopOpt in the current active module Main

In [ ]:
# Space
reffe_scalar = ReferenceFE(lagrangian,Float64,1)
V_φ = TestFESpace(model,reffe_scalar)
V_reg = TestFESpace(model,reffe_scalar;dirichlet_tags=["Gamma_N",])
U_reg = TrialFESpace(V_reg)

In [ ]:

# Cut the background model_f1((x,y,z),q,r) = - cos(q*π*x)*cos(q*π*y)*cos(q*π*z)/q - r/q
_f1((x,y,z),q,r) = - cos(q*π*x)*cos(q*π*y)*cos(q*π*z)/q - r/q
_f2((x,y,z)) = -sqrt(x^2+y^2)+0.9
φh = interpolate(x->min(_f1(x,4,0.1),_f2(x)),V_φ)